# 📊 Análise de Concorrência - ML Project

Este notebook demonstra como usar o sistema AutoML do ML Project para analisar dados de concorrência e otimizar estratégias de vendas no Mercado Livre.

## 🎯 Objetivos
- Analisar dados de produtos concorrentes
- Prever preços ótimos com AutoML
- Identificar oportunidades de mercado
- Gerar insights acionáveis

In [ ]:
# Importações necessárias
import sys
import os
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

# Importar módulos do AutoML
from automl.experiment import ExperimentManager
from automl.tuning import HyperparameterTuner
from automl.tracking import create_tracker

# Configurações
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)

print("✅ Importações realizadas com sucesso!")

## 📦 1. Geração de Dados Simulados de Concorrência

Vamos criar um dataset realista simulando dados de produtos concorrentes no Mercado Livre.

In [ ]:
# Gerar dados simulados de concorrência
np.random.seed(42)

# Parâmetros do dataset
n_produtos = 1000
categorias = ['Eletrônicos', 'Casa e Jardim', 'Moda', 'Esportes', 'Livros']
vendedores = ['Vendedor A', 'Vendedor B', 'Vendedor C', 'Nosso Vendedor', 'Concorrente Premium']

# Criar dataset de concorrência
dados_concorrencia = pd.DataFrame({
    'produto_id': [f'ML{i:06d}' for i in range(1, n_produtos + 1)],
    'categoria': np.random.choice(categorias, n_produtos),
    'vendedor': np.random.choice(vendedores, n_produtos),
    'preco': np.random.lognormal(mean=4, sigma=1, size=n_produtos).round(2),
    'vendas_30d': np.random.poisson(lam=20, size=n_produtos),
    'avaliacoes': np.random.randint(0, 1000, n_produtos),
    'nota_media': np.random.uniform(3.0, 5.0, n_produtos).round(1),
    'frete_gratis': np.random.choice([True, False], n_produtos, p=[0.7, 0.3]),
    'desconto_pct': np.random.uniform(0, 50, n_produtos).round(0),
    'posicao_busca': np.random.randint(1, 101, n_produtos),
    'clicks_30d': np.random.poisson(lam=100, size=n_produtos),
    'conversao_pct': np.random.uniform(1, 15, n_produtos).round(2)
})

# Adicionar algumas correlações realistas
# Produtos com frete grátis tendem a vender mais
dados_concorrencia.loc[dados_concorrencia['frete_gratis'], 'vendas_30d'] *= 1.3
dados_concorrencia['vendas_30d'] = dados_concorrencia['vendas_30d'].round().astype(int)

# Produtos com melhor posição tendem a ter mais clicks
dados_concorrencia['clicks_30d'] = (dados_concorrencia['clicks_30d'] * 
                                   (101 - dados_concorrencia['posicao_busca']) / 100).round().astype(int)

# Mostrar estatísticas básicas
print(f"📊 Dataset criado com {len(dados_concorrencia)} produtos")
print(f"📈 Categorias: {dados_concorrencia['categoria'].unique()}")
print(f"🏪 Vendedores: {dados_concorrencia['vendedor'].unique()}")

dados_concorrencia.head()

## 📈 2. Análise Exploratória dos Dados

In [ ]:
# Criar visualizações
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# 1. Distribuição de preços por categoria
sns.boxplot(data=dados_concorrencia, x='categoria', y='preco', ax=axes[0,0])
axes[0,0].set_title('Distribuição de Preços por Categoria')
axes[0,0].tick_params(axis='x', rotation=45)

# 2. Vendas vs Preço
sns.scatterplot(data=dados_concorrencia, x='preco', y='vendas_30d', 
                hue='categoria', alpha=0.6, ax=axes[0,1])
axes[0,1].set_title('Relação Preço vs Vendas')
axes[0,1].legend(bbox_to_anchor=(1.05, 1), loc='upper left')

# 3. Performance por vendedor
vendas_por_vendedor = dados_concorrencia.groupby('vendedor')['vendas_30d'].sum().sort_values(ascending=True)
vendas_por_vendedor.plot(kind='barh', ax=axes[1,0])
axes[1,0].set_title('Vendas Totais por Vendedor (30d)')

# 4. Impacto do frete grátis
frete_vendas = dados_concorrencia.groupby('frete_gratis')['vendas_30d'].mean()
frete_vendas.plot(kind='bar', ax=axes[1,1])
axes[1,1].set_title('Vendas Médias: Frete Grátis vs Pago')
axes[1,1].set_xticklabels(['Frete Pago', 'Frete Grátis'], rotation=0)

plt.tight_layout()
plt.show()

# Estatísticas por categoria
print("\n📊 Estatísticas por Categoria:")
estatisticas_categoria = dados_concorrencia.groupby('categoria').agg({
    'preco': ['mean', 'median', 'std'],
    'vendas_30d': ['mean', 'sum'],
    'conversao_pct': 'mean'
}).round(2)

print(estatisticas_categoria)

## 🤖 3. Experimento AutoML: Previsão de Vendas

Vamos usar o sistema AutoML para prever vendas baseado nas características dos produtos.

In [ ]:
# Preparar dados para o modelo
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

# Criar features para o modelo
dados_modelo = dados_concorrencia.copy()

# Encoding de variáveis categóricas
le_categoria = LabelEncoder()
le_vendedor = LabelEncoder()

dados_modelo['categoria_encoded'] = le_categoria.fit_transform(dados_modelo['categoria'])
dados_modelo['vendedor_encoded'] = le_vendedor.fit_transform(dados_modelo['vendedor'])
dados_modelo['frete_gratis_int'] = dados_modelo['frete_gratis'].astype(int)

# Selecionar features
features = ['preco', 'categoria_encoded', 'vendedor_encoded', 'avaliacoes', 
           'nota_media', 'frete_gratis_int', 'desconto_pct', 'posicao_busca', 
           'clicks_30d', 'conversao_pct']

X = dados_modelo[features].values
y = dados_modelo['vendas_30d'].values

print(f"✅ Dados preparados: {X.shape[0]} amostras, {X.shape[1]} features")
print(f"📊 Target (vendas): min={y.min()}, max={y.max()}, média={y.mean():.1f}")

In [ ]:
# Inicializar o gerenciador de experimentos
experiment_manager = ExperimentManager("concorrencia_analysis")

# Criar experimento
experiment_id = experiment_manager.create_experiment(
    name="Previsão de Vendas - Análise de Concorrência",
    description="Modelo para prever vendas baseado em características de produtos concorrentes",
    dataset_info={
        "shape": X.shape,
        "features": features,
        "target": "vendas_30d",
        "source": "dados_simulados_concorrencia",
        "categories": list(dados_concorrencia['categoria'].unique()),
        "date_range": "últimos_30_dias"
    }
)

print(f"🆔 Experimento criado: {experiment_id}")

In [ ]:
# Executar experimento AutoML
print("🚀 Iniciando experimento AutoML...")

results = experiment_manager.run_basic_experiment(
    experiment_id=experiment_id,
    X=X,
    y=y,
    problem_type="regression"  # Previsão de vendas é um problema de regressão
)

print(f"✅ Experimento concluído!")
print(f"🏆 Melhor modelo: {results['best_model']}")
print(f"📊 Melhor score: {results['best_score']:.4f}")

# Mostrar resultados de todos os modelos
print("\n📈 Resultados por modelo:")
for model_result in results['models_tested']:
    if 'error' in model_result:
        print(f"❌ {model_result['model_name']}: {model_result['error']}")
    else:
        print(f"✅ {model_result['model_name']}: {model_result['score']:.4f}")

## ⚙️ 4. Otimização de Hiperparâmetros

Vamos otimizar o melhor modelo encontrado.

In [ ]:
# Inicializar o otimizador de hiperparâmetros
tuner = HyperparameterTuner()

# Criar modelo baseado no melhor resultado
if results['best_model'] == 'random_forest':
    from sklearn.ensemble import RandomForestRegressor
    best_model = RandomForestRegressor(random_state=42)
    model_type = "random_forest_regressor"
else:
    from sklearn.linear_model import LinearRegression
    best_model = LinearRegression()
    model_type = "linear_regression"

print(f"🔧 Otimizando hiperparâmetros para: {model_type}")

In [ ]:
# Executar otimização de hiperparâmetros
tuning_results = tuner.auto_tune_model(
    model=best_model,
    model_type=model_type,
    X=X,
    y=y,
    method="random_search",  # Mais rápido que grid_search
    scoring="neg_mean_squared_error",  # Apropriado para regressão
    cv=3  # 3-fold CV para velocidade
)

print(f"🎯 Otimização concluída!")
print(f"🏆 Melhor score: {tuning_results['best_score']:.4f}")
print(f"⚙️ Melhores parâmetros: {tuning_results['best_params']}")
print(f"⏱️ Tempo de execução: {tuning_results['duration_seconds']:.1f} segundos")

## 📊 5. Tracking de Experimentos com MLflow

In [ ]:
# Inicializar tracker
tracker = create_tracker("concorrencia_experiments")

# Rastrear o experimento AutoML
automl_run_id = tracker.track_automl_experiment(
    experiment_results=results,
    dataset_info={
        "shape": X.shape,
        "features": features,
        "source": "concorrencia_simulada"
    }
)

# Rastrear a otimização de hiperparâmetros
tuning_run_id = tracker.track_hyperparameter_tuning(
    tuning_results=tuning_results
)

print(f"📈 Experimentos rastreados:")
print(f"  - AutoML: {automl_run_id}")
print(f"  - Tuning: {tuning_run_id}")

## 💡 6. Insights e Recomendações

Com base nos resultados do AutoML, vamos gerar insights acionáveis.

In [ ]:
# Análise de feature importance (se modelo for Random Forest)
if results['best_model'] == 'random_forest' and 'best_params' in tuning_results:
    from sklearn.ensemble import RandomForestRegressor
    
    # Treinar modelo final com melhores parâmetros
    final_model = RandomForestRegressor(**tuning_results['best_params'], random_state=42)
    final_model.fit(X, y)
    
    # Feature importance
    importance_df = pd.DataFrame({
        'feature': features,
        'importance': final_model.feature_importances_
    }).sort_values('importance', ascending=False)
    
    # Visualizar importância das features
    plt.figure(figsize=(10, 6))
    sns.barplot(data=importance_df, x='importance', y='feature')
    plt.title('Importância das Features para Previsão de Vendas')
    plt.xlabel('Importância')
    plt.tight_layout()
    plt.show()
    
    print("🔍 Features mais importantes:")
    for _, row in importance_df.head().iterrows():
        print(f"  - {row['feature']}: {row['importance']:.3f}")
else:
    print("ℹ️ Feature importance não disponível para este modelo")

In [ ]:
# Gerar recomendações específicas
print("💡 INSIGHTS E RECOMENDAÇÕES BASEADAS NO AUTOML:")
print("=" * 60)

# 1. Análise de preços por categoria
preco_categoria = dados_concorrencia.groupby('categoria').agg({
    'preco': ['mean', 'median'],
    'vendas_30d': 'mean',
    'conversao_pct': 'mean'
}).round(2)

print("\n📊 1. ANÁLISE DE PREÇOS POR CATEGORIA:")
for categoria in dados_concorrencia['categoria'].unique():
    dados_cat = dados_concorrencia[dados_concorrencia['categoria'] == categoria]
    preco_medio = dados_cat['preco'].mean()
    vendas_medias = dados_cat['vendas_30d'].mean()
    print(f"  • {categoria}: Preço médio R$ {preco_medio:.2f}, Vendas médias {vendas_medias:.1f}")

# 2. Impacto do frete grátis
frete_gratis_vendas = dados_concorrencia[dados_concorrencia['frete_gratis']]['vendas_30d'].mean()
frete_pago_vendas = dados_concorrencia[~dados_concorrencia['frete_gratis']]['vendas_30d'].mean()
aumento_frete = ((frete_gratis_vendas - frete_pago_vendas) / frete_pago_vendas) * 100

print(f"\n🚚 2. IMPACTO DO FRETE GRÁTIS:")
print(f"  • Produtos com frete grátis vendem {aumento_frete:.1f}% mais")
print(f"  • Vendas médias: Frete grátis {frete_gratis_vendas:.1f} vs Frete pago {frete_pago_vendas:.1f}")

# 3. Posicionamento na busca
posicao_top10 = dados_concorrencia[dados_concorrencia['posicao_busca'] <= 10]['vendas_30d'].mean()
posicao_outros = dados_concorrencia[dados_concorrencia['posicao_busca'] > 10]['vendas_30d'].mean()

print(f"\n🔍 3. IMPORTÂNCIA DO POSICIONAMENTO:")
print(f"  • Top 10 posições: {posicao_top10:.1f} vendas médias")
print(f"  • Outras posições: {posicao_outros:.1f} vendas médias")
print(f"  • Diferença: {((posicao_top10 - posicao_outros) / posicao_outros * 100):.1f}%")

# 4. Recomendações acionáveis
print(f"\n🎯 4. RECOMENDAÇÕES ACIONÁVEIS:")
print(f"  ✅ Implementar frete grátis quando possível (+{aumento_frete:.1f}% vendas)")
print(f"  ✅ Focar em melhorar posicionamento SEO/busca")
print(f"  ✅ Monitorar preços da concorrência por categoria")
print(f"  ✅ Investir em avaliações e nota dos produtos")
print(f"  ✅ Usar modelo AutoML para precificação dinâmica")

# 5. Próximos passos
print(f"\n🚀 5. PRÓXIMOS PASSOS:")
print(f"  • Implementar monitoramento automático de preços")
print(f"  • Integrar modelo com sistema de precificação")
print(f"  • Coletar dados reais da API do Mercado Livre")
print(f"  • Retreinar modelo mensalmente")
print(f"  • A/B testing de estratégias de pricing")

## 📋 7. Relatório Final do Experimento

In [ ]:
# Gerar relatório completo
report = experiment_manager.generate_experiment_report(experiment_id)
print(report)

# Salvar relatório
with open(f'relatorio_concorrencia_{experiment_id}.md', 'w', encoding='utf-8') as f:
    f.write(report)
    
print(f"\n💾 Relatório salvo como: relatorio_concorrencia_{experiment_id}.md")

## 🎉 Conclusão

Este notebook demonstrou como usar o sistema AutoML do ML Project para:

1. **📊 Análisar dados de concorrência** de forma automatizada
2. **🤖 Aplicar AutoML** para previsão de vendas
3. **⚙️ Otimizar hiperparâmetros** automaticamente
4. **📈 Rastrear experimentos** com MLflow
5. **💡 Gerar insights acionáveis** para estratégia de vendas

### Próximos Passos
- Integrar com dados reais da API do Mercado Livre
- Implementar pipeline de retreinamento automático
- Criar dashboard de monitoramento
- A/B testing de estratégias derivadas dos insights

---

**Desenvolvido pelo ML Project Team** 🚀